### Mariam Gasoyan
# Homework 3

In [ ]:
import re
import time
import requests
#import numpy as np
import pandas as pd
from textblob import TextBlob, Word
from scrapy.http import TextResponse
import nltk
from nltk.corpus import stopwords, wordnet
#from nltk.tokenize import sent_tokenize, word_tokenize
#from nltk.stem import PorterStemmer, WordNetLemmatizer
#from sklearn.feature_extraction.text import TfidfVectorizer
#from selenium import webdriver
#import selenium.webdriver.common.keys as Keys

###  1.  Scrape all the latest news headings (and publication time) from https://www.tert.am/en and answer the following questions:

In [ ]:
url = "https://www.tert.am/en"

In [ ]:
response = requests.get(url)
response.status_code

In [ ]:
response = TextResponse(url = response.url, body = response.text, encoding = "utf-8")
latest_news = response.xpath('//ul[@class="list list--with-scroll scroller-block show-link-visited"]//span[@class="list__title db"]/text()').extract()
print(len(latest_news))
latest_news[:5]

a) What are the top most frequent words? (not stopword)

In [ ]:
sw = stopwords.words("english")
word_list = []
for sentense in latest_news:
    sentense = sentense.replace("'", "").replace("‘", "").replace("’", "").replace("–", " ").replace(".", "")
    sentense=TextBlob(sentense)
    sentense.words = sentense.words.lemmatize()
    for word in sentense.words:
        word = word.lemmatize("v").lower()        
        if word not in sw:
            word_list.append(word)

In [ ]:
df = pd.DataFrame(word_list)
print("Top most frequent words: \n------------------------")
df[0].value_counts()[:10]

 b) How many typical Armenian surnames can be counted in titles? (assume surname in
any word starting with an uppercase and ending with “yan”),


In [ ]:
surnames = []
for sentense in latest_news:
    surnames.extend(re.findall('[A-Z][a-z]+yan', sentense))
print(len(surnames), 'surnames:', surnames)

c) In what part of the day are most of the articles published? (you may provide some
range such as from 10am to 11am).


In [ ]:
times_draft = response.xpath('//ul[@class="list list--with-scroll scroller-block show-link-visited"]//span[@class="list__date db fb fs14"]/text()').extract()
times_draft[:5]

In [ ]:
times = []
for t in times_draft:
    times.append(int(re.findall('\d\d:', t)[0].replace(':', '')))
    times_df = pd.DataFrame(times)

In [ ]:
times_dist = nltk.FreqDist(times)

In [ ]:
common_hours = nltk.Counter(times_dist).most_common()
common_hours[:3]

In [ ]:
print("Most common hours:", str(common_hours[0][0])+":00-" + str(common_hours[0][0]+1) +":00,", common_hours[0][1], "times.")

### 2) Scrape all jobs and companies from https://staff.am/en/jobs and answer the following questions:

In [ ]:
all_urls = ["https://staff.am/en/jobs?page={}&per-page=50".format(i) for i in range(1,14)]

In [ ]:
requests.get(all_urls[0]).status_code

In [ ]:
def job_scraper(url):
    response = requests.get(url)
    response = TextResponse(url = response.url, body = response.text, encoding = "utf-8")
    jobs = response.xpath('//p[@class="font_bold"]/text()').extract()
    return jobs

all_jobs = []
for url in all_urls:
    all_jobs.extend(job_scraper(url))
    
len(all_jobs)

In [ ]:
def company_scraper(url):
    response = requests.get(url)
    response = TextResponse(url = response.url, body = response.text, encoding = "utf-8")
    companies = response.xpath('//p[@class="job_list_company_title"]/text()').extract()
    return companies

all_companies = []
for url in all_urls:
    all_companies.extend(company_scraper(url))
    
len(all_companies)

In [ ]:
len(all_jobs) == len(all_companies)

a) Which company has the highest number of job postings?

In [ ]:
companies_df = pd.DataFrame(all_companies)

In [ ]:
top_company = list(dict(companies_df[0].value_counts()[:1]).keys())[0]
print("The top company:", top_company)

b) How many postings does this company have?

In [ ]:
print(top_company, 'has', companies_df[0].value_counts()[0], 'jobs posted.')

c) Which company has the highest number of “developer” (or related) job postings?

In [ ]:
dev_list_indices = []
for i in range(len(all_jobs)):
    dev = re.findall(".eveloper[a-z]*", all_jobs[i])
    if len(dev) >0:
        dev_list_indices.append(i)

In [ ]:
dev_companies = []
for i in dev_list_indices:
    dev_companies.append(all_companies[i])

In [ ]:
dev_companies_df = pd.DataFrame(dev_companies)
top_dev_company = list(dict(dev_companies_df[0].value_counts()[:1]).keys())[0]
print(top_dev_company, "has the highest number of “developer” (or related) job postings")

### 3) Scrape the table with exchange rates from www.rates.am for the first 2 weeks of July 
#### (can be done both with and without Selenium).

In [ ]:
response = requests.get('http://rates.am/en')
response.status_code

In [ ]:
days = []
for i in range(2, 16):
    if i < 10:
        days.append("0" + str(i))
    else:
        days.append(str(i))

as there was no specification in the assignment, I'm going to scrape the cash exchange rates of all exchange points

In [ ]:
all_ex_urls = []
for d in days:
    all_ex_urls.append('http://rates.am/en/armenian-dram-exchange-rates/exchange-points/cash/2019/07/'+d+'/20-00')

In [ ]:
averages = []
for url in all_ex_urls:
    table = pd.read_html(url)[3]
    buy_list =list(pd.to_numeric(table[5].dropna()[2:-1]))
    sell_list =list(pd.to_numeric(table[5].dropna()[2:-1]))
    avg = (sum(buy_list) + sum(buy_list))/len(buy_list)/2
    averages.append(avg)

a) What is the average USD rate for this duration?

In [ ]:
average_rate = sum(averages)/len(averages)

In [ ]:
print("Average rate:", average_rate)

b) Plot the average daily USD rate over time.


In [ ]:
avg_rates_df = pd.DataFrame(averages, days) 

In [ ]:
avg_rates_df.plot()